In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
import datetime
import matplotlib.pyplot as plt
import collections as cc
import seaborn as sns
from collections import Counter
import re
import spacy

In [30]:
data_dir='0819_keyword_analysis.csv'
data=pd.read_csv(data_dir,encoding='utf-8')

In [31]:
data_desc=data[['id','name','keyword','price','flag_taekpo','flag_used','flag_exchg','description','image_count','name_kor','total_view','purchase_duration']]
data_nodesc=data[['id','name','keyword','price','flag_taekpo','flag_used','flag_exchg','image_count','name_kor','total_view','purchase_duration']]

In [32]:
air_force=data_nodesc[data_nodesc['name_kor']=="나이키 에어포스 1 '07 로우 화이트"]
air_force=air_force[['id','name']]

In [33]:
air_force.to_csv("airforce_only.csv",index=False,encoding='utf-8')

## 한글 영어 번역 -> Fairseq

In [34]:
spm_train --input=train.ko --model_prefix=korean --vocab_size=32000 --character_coverage=1.0
spm_train --input=train.en --model_prefix=english --vocab_size=32000 --character_coverage=1.0

SyntaxError: cannot assign to expression (2588148197.py, line 1)

## 한글 불용어 제거

In [8]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer


In [22]:
texts = air_force['name'].fillna('')
keywords=air_force['keyword'].fillna('')

In [25]:
okt = Okt()
# 조사와 접속사를 제거한 형태소 리스트 생성
stopwords = ['에어포스', '나이키', '에어', '포스', '남성', '우먼', '여성', '제']

# 텍스트 데이터를 형태소 분석하여 토큰화
def tokenize(text):
    tokens = okt.morphs(text, stem=True)  # 형태소 분석 및 어간 추출
    tokens = [word for word in tokens if word not in stopwords]  # 불용어 제거
    return ' '.join(tokens)

# 텍스트 데이터에 대해 토큰화 수행
tokenized_texts = texts.apply(tokenize)


In [26]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(tokenized_texts)

# 주요 구문 추출
features = vectorizer.get_feature_names_out()
# TF-IDF 가중치 높은 상위 n개의 구문 추출
top_n = 30
tfidf_scores = X.toarray().sum(axis=0)
top_features = [features[i] for i in tfidf_scores.argsort()[-top_n:]]

print("Top Features:", top_features)

Top Features: ['신발', '제품', '225', '흰색', '트리플', '스니커즈', '230', '파다', '희다', '오다', '정품', '290', '235', '285', 'mm', '255', '245', '250', '운동화', '240', '260', '상품', '사이즈', '280', '275', '270', '07', '265', '화이트', '로우']
